# 라이브러리

In [46]:
options(warn =-1, repr.plot.width=15, repr.plot.height=9)
# ## 패키지 설치
library(dplyr)
library(stringr)
library(ggplot2)
library(gridExtra)
library(scales)
library(randomForest)
library(lubridate)
library(caret)
library(e1071)
library(nnet)
library(class)


# 전처리

In [47]:
### .csv 불러오기
data <- read.csv("./data/refined_data_filtered_major5_subset.csv")

In [48]:
data$근무지역분류 <- as.character(data$근무지역분류)
data$근무지역분류 <- factor(data$근무지역분류)
data$장애유형 <- as.character(data$장애유형)
data$장애유형 <- factor(data$장애유형)
data$중증여부 <- as.character(data$중증여부)
data$중증여부 <- factor(data$중증여부)
data$근무지역 <- as.character(data$근무지역)
data$근무지역 <- factor(data$근무지역)
data$취업직종대분류 <- as.character(data$취업직종대분류)
data$취업직종대분류 <- factor(data$취업직종대분류)
data$연령대 <- as.character(data$연령대)
data$연령대 <- factor(data$연령대)
data$장애유형_중증여부 <- as.character(data$장애유형_중증여부)
data$장애유형_중증여부 <- factor(data$장애유형_중증여부)
data$월 <- as.character(data$월)
data$월 <- factor(data$월)

In [49]:
data$target<-(ifelse(data$취업직종대분류=="경영·행정·사무직","경영·행정·사무직",data$취업직종대분류))

data$target <- ifelse(data$취업직종대분류 %in% c("청소 및 기타 개인서비스직", "음식 서비스직", "미용·예식 서비스직", "영업·판매직", "스포츠·레크리에이션직","경호·경비직"), "서비스직", data$target)

data$target <- ifelse(grepl("^\\d+$", data$target), "기타", data$target)

In [50]:
str(data)

'data.frame':	24979 obs. of  11 variables:
 $ 취업일자         : chr  "2020-01-01" "2020-01-01" "2020-01-01" "2020-01-01" ...
 $ 연령             : int  65 61 53 64 53 60 63 40 56 62 ...
 $ 장애유형         : Factor w/ 5 levels "시각장애","정신장애",..: 1 1 1 1 1 1 1 1 4 1 ...
 $ 중증여부         : Factor w/ 2 levels "경증","중증": 2 2 1 2 1 2 2 2 2 2 ...
 $ 근무지역         : Factor w/ 270 levels "강원 강릉시",..: 97 99 114 106 101 111 108 119 18 17 ...
 $ 취업직종대분류   : Factor w/ 21 levels "건설·채굴직",..: 2 2 2 2 2 2 2 9 9 9 ...
 $ 연령대           : Factor w/ 6 levels "1","2","3","4",..: 5 5 4 5 4 5 5 3 4 5 ...
 $ 장애유형_중증여부: Factor w/ 10 levels "시각장애-경증",..: 2 2 1 2 1 2 2 2 8 2 ...
 $ 월               : Factor w/ 12 levels "1","10","11",..: 1 1 1 1 1 1 5 6 10 1 ...
 $ 근무지역분류     : Factor w/ 17 levels "강원","경기",..: 4 4 4 4 4 4 4 4 1 1 ...
 $ target           : chr  "경영·행정·사무직" "경영·행정·사무직" "경영·행정·사무직" "경영·행정·사무직" ...


In [51]:
# 불필요한 컬럼 제거

data_filtered <- subset(data, select = -c(취업일자,연령,장애유형,중증여부,근무지역))
sum(is.na(data_filtered))

[1] 0

In [52]:
# 연령대
data_filtered$age_20 <- data_filtered$연령대 == 1
data_filtered$age_30 <- data_filtered$연령대 == 2
data_filtered$age_40 <- data_filtered$연령대 == 3
data_filtered$age_50 <- data_filtered$연령대 == 4
data_filtered$age_60 <- data_filtered$연령대 == 5
data_filtered$age_70 <- data_filtered$연령대 == 6

In [53]:
# 장애유형_중증여부 바꾸기
data_filtered$시각장애_중증 <- data_filtered$장애유형_중증여부 == "시각장애-중증"
data_filtered$시각장애_경증 <- data_filtered$장애유형_중증여부 == "시각장애-경증"

data_filtered$지체장애_중증 <- data_filtered$장애유형_중증여부 == "지체장애-중증"
data_filtered$지체장애_경증 <- data_filtered$장애유형_중증여부 == "지체장애-경증"

data_filtered$지적장애_경증 <- data_filtered$장애유형_중증여부 == "지적장애-경증"
data_filtered$지적장애_중증 <- data_filtered$장애유형_중증여부 == "지적장애-중증"

data_filtered$청각장애_경증 <- data_filtered$장애유형_중증여부 == "청각장애-경증"
data_filtered$청각장애_중증 <- data_filtered$장애유형_중증여부 == "청각장애-중증"

data_filtered$정신장애_경증 <- data_filtered$장애유형_중증여부 == "정신장애-경증"
data_filtered$정신장애_중증 <- data_filtered$장애유형_중증여부 == "정신장애-중증"

In [54]:
# 근무지역 top 6 남기기 서울 경기 경남 부산 인천 경북 
data_filtered <- data_filtered %>%
  filter(근무지역분류 %in% c("서울","경기","인천"))

In [55]:
#   근무지역 분류 바꾸기

data_filtered$서울 <- data_filtered$근무지역분류 == "서울"
data_filtered$경기 <- data_filtered$근무지역분류 == "경기"
data_filtered$인천 <- data_filtered$근무지역분류 == "인천"


In [56]:
# 월 top6
data_filtered <- data_filtered %>%
  filter(월 %in% c("1","6","7","9","8","3"))

In [57]:
# 월
data_filtered$Jan <- data_filtered$월  == 01

data_filtered$Mar <- data_filtered$월  == 03

data_filtered$Jun <- data_filtered$월  == 06
data_filtered$Jul <- data_filtered$월  == 07
data_filtered$Aug <- data_filtered$월  == 08
data_filtered$Sep <- data_filtered$월  == 09


In [58]:
str(data_filtered)
sum(is.na(data_filtered))

'data.frame':	7475 obs. of  31 variables:
 $ 취업직종대분류   : Factor w/ 21 levels "건설·채굴직",..: 9 9 9 9 9 9 9 9 9 9 ...
 $ 연령대           : Factor w/ 6 levels "1","2","3","4",..: 1 1 2 1 1 1 1 1 1 4 ...
 $ 장애유형_중증여부: Factor w/ 10 levels "시각장애-경증",..: 6 6 6 4 6 6 10 6 6 1 ...
 $ 월               : Factor w/ 12 levels "1","10","11",..: 6 6 6 6 6 6 6 6 9 9 ...
 $ 근무지역분류     : Factor w/ 17 levels "강원","경기",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ target           : chr  "기타" "기타" "기타" "기타" ...
 $ age_20           : logi  TRUE TRUE FALSE TRUE TRUE TRUE ...
 $ age_30           : logi  FALSE FALSE TRUE FALSE FALSE FALSE ...
 $ age_40           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_50           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_60           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_70           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_중증    : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_경증    : logi  FALSE FALSE FALSE FALSE FALSE FALSE .

[1] 0

In [59]:
str(data_filtered)
sum(is.na(data_filtered))

'data.frame':	7475 obs. of  31 variables:
 $ 취업직종대분류   : Factor w/ 21 levels "건설·채굴직",..: 9 9 9 9 9 9 9 9 9 9 ...
 $ 연령대           : Factor w/ 6 levels "1","2","3","4",..: 1 1 2 1 1 1 1 1 1 4 ...
 $ 장애유형_중증여부: Factor w/ 10 levels "시각장애-경증",..: 6 6 6 4 6 6 10 6 6 1 ...
 $ 월               : Factor w/ 12 levels "1","10","11",..: 6 6 6 6 6 6 6 6 9 9 ...
 $ 근무지역분류     : Factor w/ 17 levels "강원","경기",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ target           : chr  "기타" "기타" "기타" "기타" ...
 $ age_20           : logi  TRUE TRUE FALSE TRUE TRUE TRUE ...
 $ age_30           : logi  FALSE FALSE TRUE FALSE FALSE FALSE ...
 $ age_40           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_50           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_60           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_70           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_중증    : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_경증    : logi  FALSE FALSE FALSE FALSE FALSE FALSE .

[1] 0

In [60]:
# 불필요한 컬럼 제거
data_filtered_subset <- subset(data_filtered, select = -c(연령대,장애유형_중증여부,월,근무지역분류))

In [61]:
str(data_filtered_subset)
sum(is.na(data_filtered_subset))

'data.frame':	7475 obs. of  27 variables:
 $ 취업직종대분류: Factor w/ 21 levels "건설·채굴직",..: 9 9 9 9 9 9 9 9 9 9 ...
 $ target        : chr  "기타" "기타" "기타" "기타" ...
 $ age_20        : logi  TRUE TRUE FALSE TRUE TRUE TRUE ...
 $ age_30        : logi  FALSE FALSE TRUE FALSE FALSE FALSE ...
 $ age_40        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_50        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_60        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_70        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_중증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지체장애_중증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지체장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지적장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지적장애_중증 : logi  TRUE TRUE TRUE FALSE TRUE TRUE ...
 $ 청각장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 청각장애_중증 : logi  FALSE FALSE FALSE FALSE FALSE 

[1] 0

In [62]:
# 불필요한 컬럼 제거
data_filtered_subset <- subset(data_filtered, select = -c(연령대,장애유형_중증여부,월,근무지역분류))

In [63]:
str(data_filtered_subset)

'data.frame':	7475 obs. of  27 variables:
 $ 취업직종대분류: Factor w/ 21 levels "건설·채굴직",..: 9 9 9 9 9 9 9 9 9 9 ...
 $ target        : chr  "기타" "기타" "기타" "기타" ...
 $ age_20        : logi  TRUE TRUE FALSE TRUE TRUE TRUE ...
 $ age_30        : logi  FALSE FALSE TRUE FALSE FALSE FALSE ...
 $ age_40        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_50        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_60        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_70        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_중증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지체장애_중증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지체장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지적장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지적장애_중증 : logi  TRUE TRUE TRUE FALSE TRUE TRUE ...
 $ 청각장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 청각장애_중증 : logi  FALSE FALSE FALSE FALSE FALSE 

# 변동사항

In [64]:
data_filtered_subset$target <- factor(data_filtered_subset$target)

In [65]:
data_filtered_subset <- subset(data_filtered_subset, select = -c(취업직종대분류))

In [66]:
str(data_filtered_subset)
summary(data_filtered_subset)

'data.frame':	7475 obs. of  26 variables:
 $ target       : Factor w/ 3 levels "경영·행정·사무직",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ age_20       : logi  TRUE TRUE FALSE TRUE TRUE TRUE ...
 $ age_30       : logi  FALSE FALSE TRUE FALSE FALSE FALSE ...
 $ age_40       : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_50       : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_60       : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_70       : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_중증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_경증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지체장애_중증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지체장애_경증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지적장애_경증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지적장애_중증: logi  TRUE TRUE TRUE FALSE TRUE TRUE ...
 $ 청각장애_경증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 청각장애_중증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 정신장애_경증: logi  FALSE FALSE FALSE FALSE FA

              target       age_20          age_30          age_40       
 경영·행정·사무직:2787   Mode :logical   Mode :logical   Mode :logical  
 기타            :1882   FALSE:5155      FALSE:6276      FALSE:6407     
 서비스직        :2806   TRUE :2320      TRUE :1199      TRUE :1068     
   age_50          age_60          age_70        시각장애_중증  
 Mode :logical   Mode :logical   Mode :logical   Mode :logical  
 FALSE:6305      FALSE:6217      FALSE:7015      FALSE:6835     
 TRUE :1170      TRUE :1258      TRUE :460       TRUE :640      
 시각장애_경증   지체장애_중증   지체장애_경증   지적장애_경증  
 Mode :logical   Mode :logical   Mode :logical   Mode :logical  
 FALSE:7023      FALSE:6575      FALSE:5749      FALSE:7471     
 TRUE :452       TRUE :900       TRUE :1726      TRUE :4        
 지적장애_중증   청각장애_경증   청각장애_중증   정신장애_경증  
 Mode :logical   Mode :logical   Mode :logical   Mode :logical  
 FALSE:4986      FALSE:7166      FALSE:6972      FALSE:7473     
 TRUE :2489      TRUE :309       TRUE :503       TRUE :2    

# 시험준비

In [67]:
samp <- createDataPartition(data_filtered_subset$target, p=0.7, list=F)

#   학습용
data.tr <- data_filtered_subset[samp,]

#   검증용
data.test <- data_filtered_subset[-samp,]

In [68]:
#   KNN을 위한 Training Data 준비
data.training <- subset(data.tr, select = -target)
data.training.label <- data.tr$target

In [69]:
#   문제 및 정답 만들기
x   <- subset(data.test, select = -target)
y   <- data.test$target


# 학습시키기

In [70]:
#  랜덤포레스트 학습 시키기
rf <- randomForest(target ~., data.tr, ntree=500)

In [71]:
# SVM 학습시키기
modelsvm <- svm(target~.,data.tr)

In [72]:
# KNN 학습시키기 3,5,7,
knnmodel <- knn(train=data.training, test=x,cl=data.training.label,k=3)

In [73]:
# nnet 학습시키기
modelnnet <- nnet(target~.,data.tr,size =20)

# weights:  583
initial  value 6133.543822 
iter  10 value 4889.045246
iter  20 value 4743.032082
iter  30 value 4606.347748
iter  40 value 4542.007166
iter  50 value 4501.705340
iter  60 value 4471.135947
iter  70 value 4428.779769
iter  80 value 4395.150338
iter  90 value 4372.751095
iter 100 value 4355.221924
final  value 4355.221924 
stopped after 100 iterations


# 검증하기

In [74]:
#  Random Forest 검증하기
predrf <- predict(rf, x)

In [75]:
#  SVM 검증하기
predmodelsvm <- predict(modelsvm, x)

In [76]:
# KNN 검증하기
mean(knnmodel==y) 

[1] 0.5555556

In [77]:
# nnet 검증하기
prednnet = predict(modelnnet, x,type = "class")    # class : 객관식, raw : 주관식

# 예측력 확인

In [78]:
#  RF 예측력
mean(predrf == y)

[1] 0.577867

In [79]:
#  SVM 예측력
mean(predmodelsvm == y)

[1] 0.5555556

In [80]:
#  KNN 예측력
mean(knnmodel == y)

[1] 0.5555556

In [81]:
#  nnet 예측력
mean(prednnet == y)

[1] 0.5604641

# 오분류표

In [82]:
#  Random Forest 오분류표
table(predrf,y)

                  y
predrf             경영·행정·사무직 기타 서비스직
  경영·행정·사무직              556  134      282
  기타                           95  273       93
  서비스직                      185  157      466

In [83]:
#  SVM 오분류표
table(predmodelsvm,y)

                  y
predmodelsvm       경영·행정·사무직 기타 서비스직
  경영·행정·사무직              465  111      231
  기타                           97  228       58
  서비스직                      274  225      552

In [84]:
#  KNN 오분류표
# table(y, knnmodel)

In [85]:
# nnet 오분류표
table(prednnet,y)

                  y
prednnet           경영·행정·사무직 기타 서비스직
  경영·행정·사무직              519  126      267
  기타                           97  270      107
  서비스직                      220  168      467

In [86]:
# 새로운 데이터에 대한 예측
predict(rf, (list(age_20=FALSE,age_30=FALSE,age_40=FALSE,age_50=FALSE,age_60=TRUE,age_70=FALSE,
            시각장애_중증=TRUE,시각장애_경증=FALSE,지체장애_경증=FALSE,지체장애_중증=FALSE,지적장애_경증=FALSE,지적장애_중증=FALSE,
            청각장애_경증=FALSE,청각장애_중증=FALSE,정신장애_경증=FALSE,정신장애_중증=FALSE,
            경북=TRUE,강원=FALSE,전남=FALSE,충북=FALSE,경기=FALSE,인천=FALSE,서울=FALSE,울산=FALSE,대전=FALSE,부산=FALSE,
            전북=FALSE,광주=FALSE,충남=FALSE,대구=FALSE,경남=FALSE,제주=FALSE,세종=FALSE,경북=FALSE,
            Jan=TRUE,Feb=FALSE,Mar=FALSE,Apr=FALSE,May=FALSE,Jun=FALSE,Jul=FALSE,Aug=FALSE,Sep=FALSE,
            Oct=FALSE,Nov=FALSE,Dec=FALSE)), type = "prob")

# pred_prob <- predict(rf, new_data, type = "prob")
# pred_prob <- predict(rf, new_data, type = "prob")

,경영·행정·사무직,기타,서비스직
1,0.03,0.948,0.022
